In [7]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz

In [8]:
df_strategy = pd.read_csv('Data/ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ.csv')
df_strategy['source'] = 'ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ'

df_finance = pd.read_csv('Data/ฝ่ายการเงิน.csv')
df_finance['source'] = 'ฝ่ายการเงิน'

df_customer = pd.read_csv('Data/ฝ่ายบริหารลูกค้า.csv')
df_customer['source'] = 'ฝ่ายบริหารลูกค้า'

In [9]:
def clean_phone(phone):
    phone = str(phone)
    return re.sub(r'\D', '', phone)

def clean_email(email):
    return str(email).strip().lower()

def split_name(name):
    parts = str(name).split()
    if len(parts) == 2:
        return parts[0], parts[1]
    return name, ''

df_customer[['firstname', 'lastname']] = df_customer['name'].apply(lambda x: pd.Series(split_name(x)))
df_customer['phone'] = df_customer['phone'].apply(clean_phone)
df_customer['email'] = df_customer['email'].apply(clean_email)
df_customer['address'] = df_customer['address'].astype(str).str.strip()
df_customer['customer_type'] = df_customer['customer_type'].astype(str).str.strip().str.lower()
df_customer['company'] = df_customer['company'].astype(str).str.strip().str.lower()

for df in [df_strategy, df_finance]:
    df['phone'] = df['phone'].apply(clean_phone)
    df['email'] = df['email'].apply(clean_email)
    df['firstname'] = df['firstname'].astype(str).str.strip()
    df['lastname'] = df['lastname'].astype(str).str.strip()
    df['address'] = df['address'].astype(str).str.strip()
    df['customer_type'] = df['customer_type'].astype(str).str.strip().str.lower()
    df['company'] = df['company'].astype(str).str.strip().str.lower()


In [10]:
def normalize_customer_type(val):
    if 'อุตสาหกรรม' in val or 'industry' in val:
        return 'industry'
    if 'บ้าน' in val or 'home' in val:
        return 'home'
    return val

for df in [df_strategy, df_finance, df_customer]:
    df['customer_type'] = df['customer_type'].apply(normalize_customer_type)

In [11]:
all_records = pd.concat([
    df_strategy[['firstname', 'lastname', 'phone', 'email', 'address', 'customer_type', 'company', 'source']],
    df_finance[['firstname', 'lastname', 'phone', 'email', 'address', 'customer_type', 'company', 'source']],
    df_customer[['firstname', 'lastname', 'phone', 'email', 'address', 'customer_type', 'company', 'source']]
], ignore_index=True)

def find_matches(df, threshold=90):
    matched = []
    used = set()
    for idx, row in df.iterrows():
        if idx in used:
            continue
        group = [idx]
        for idx2, row2 in df.iloc[idx+1:].iterrows():
            if idx2 in used:
                continue
            score_name = fuzz.ratio(f"{row['firstname']} {row['lastname']}", f"{row2['firstname']} {row2['lastname']}")
            score_phone = fuzz.ratio(row['phone'], row2['phone'])
            score_email = fuzz.ratio(row['email'], row2['email'])
            if (score_name > threshold or score_phone > threshold or score_email > threshold):
                group.append(idx2)
                used.add(idx2)
        matched.append(group)
        used.add(idx)
    return matched

matches = find_matches(all_records)

In [12]:
golden_records = []
for group in matches:
    group_df = all_records.iloc[group]
    record = {}
    confidence_scores = []
    sources = group_df['source'].unique().tolist()
    for col in all_records.columns:
        if col == 'source':
            continue
        vals = group_df[col].dropna().astype(str)
        if len(vals) == 0:
            record[col] = ''
            confidence_scores.append(0)
        else:
            most_common = vals.value_counts()
            record[col] = most_common.index[0] if most_common.iloc[0] > 1 else max(vals, key=len)
            confidence = most_common.iloc[0] / len(vals)
            confidence_scores.append(confidence)
    record['confidence_ratio'] = round(np.mean(confidence_scores), 2)
    record['sources'] = ','.join(sources)
    golden_records.append(record)

golden_df = pd.DataFrame(golden_records)

In [13]:
golden_df.to_csv('golden_record.csv', index=False)
golden_df.head(10)

,firstname,lastname,phone,email,address,customer_type,company,confidence_ratio,sources
0,เปมิกา,จันทร์เขียว,0946622099,pemikajunkheav@gmail.com,34/50 หมู่บ้านปรัชญาไพรเวทโฮม แยก14-14 ซ.นาคนิ...,home,-,0.86,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
1,ปณิชา,เตชะเศรษฐกุล,0978760453,panichataechasettakul@gmail.com,คอนโดแอสปาย สาทร-ราชพฤกษ์ 99/858 ถ.ราชพฤกษ์ ปา...,home,-,0.86,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
2,อาภารัตน์,พูลสวัสดิ์,0830814804,i863app@gmail.com,55/65 หมู่บ้านพลัสซิตี้พาร์ค ซ.สุคนธสวัสดิ์3 แ...,home,-,0.76,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
3,ณิชกานต์,เจนจิตรานนท์,0956784566,q98nikan@gmail.com,443 ลาดพร้าววังหิน 57 เขตลาดพร้าว แขวงลาดพร้าว...,home,-,0.86,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
4,ชัญญานุช,เจริญพนารัตน์,0987463641,maychych@homemee.com,188/12 หมู่ 3 หมู่บ้านอรรีญาเพลส ต.บางศรีเมือง...,industry,homemee,0.81,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
5,ลลิลธร,วิยา,0983816980,larin@gmail.com,D407 haus private residences 59/5 ม.17 ต.คลอ...,home,-,0.86,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
6,ศศิชา,พุฒิลือชา,025467888,sasi@booboo.com,เลขที่ 18 หมู่ที่ 6 ต.ตะคร้ำเอน อ.ท่ามะกา จ.ก...,industry,booboo,0.86,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
7,ชัญญานุช,งามพักตร์,020995448,chann@thailove.com,73 กาญจนาภิเษก 11/1 แขวงคันนายาว เขตคันนายาว ก...,industry,thailove,0.86,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
8,นวกร,เกียรติไกรกังวาน,0863337456,n998korn@gmail.com,21/925 ซ.นวมินทร์ 42 แยก 11 คลองกุ่ม บึงกุ่ม ก...,home,-,0.86,"ฝ่ายกลยุทธ์ดิจิทัลและบริหารจัดการ,ฝ่ายการเงิน,..."
